# Field Data Manipulation

The goal of this notebook is to create a file that contains all the field information that will be needed for the next folder (Correlation and Regression Manipulation). Essentially making sure that All our eggs are in one basket.

Here we will be extracting data from some of the field data spreadsheets along with calculating a couple data points.

First, we want to extract all the data that we can from the files that we are given and merging it into one file. From this point on, the linking feature that we will be using is the field NPDID number. We chose this number as opposed to just using the field name because some of the norwegian characters did not translate nicely to ASCII characters and because some of the files did not contain the field name, but all of them did contain the NPDID.

In [1]:
import numpy as np
import pandas as pd
import math

from pandas import read_excel
import seaborn as sns; sns.set(style="ticks", color_codes=True)

# Change this according to your machine directory/path
extension = "Field_Data\\"

In [2]:
# Here we are taking two files and merging them together according to their
# field name.

file_name = extension + "field_in_place_volumes.xlsx"
temp_file_1 = read_excel(file_name)
temp_file_1 = temp_file_1.drop(["Unnamed: 0", "Date sync NPD", "Updated date"], axis=1)
temp_file_1 = temp_file_1.set_index("Field name")

file_name = extension + "field_reserves.xlsx"
temp_file_2 = read_excel(file_name)
temp_file_2 = temp_file_2.drop(["Unnamed: 0", "Updated date", "NPDID field", "Date sync NPD"], axis=1)
temp_file_2 = temp_file_2.set_index("Field name")


In [3]:
# Now we are creating the main file that will contain everything.
main_file = temp_file_1.join(temp_file_2)

# Because we want to have the identifying information at the beginning of the file,
# we do some house keeping to ensure that.
cols = list(main_file)
cols[4], cols[0] = cols[0], cols[4]
main_file = main_file.ix[:,cols]

main_file

C:\Users\lawso\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\lawso\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


,NPDID field,Orig. inplace ass. liquid \n[mill Sm3],Orig. inplace ass. gas \n[bill Sm3],Orig. inplace free gas \n[bill Sm3],Orig. inplace oil \n[mill Sm3],Orig. recoverable oil \n[mill Sm3],Orig. recoverable gas \n[bill Sm3],Orig. recoverable NGL \n[mill tonn],Orig. recoverable cond. \n[mill Sm3],Orig. recoverable oil eq. \n[mill Sm3 o.e],Remaining oil \n[mill Sm3],Remaining gas \n[bill Sm3],Remaining NGL \n[mill tonn],Remaining cond. \n[mill Sm3],Remaining oil eq. \n[mill Sm3 o.e]
Field name,,,,,,,,,,,,,,,
ALBUSKJELL,43437,0,64,0.00,56,7.35,15.53,0.99,0.00,24.761,0.00,0.00,0.00,0.00,0.000
ALVE,4444332,3,1,13.50,3,2.39,9.08,1.41,0.00,14.149,0.34,2.10,0.28,0.00,2.972
ALVHEIM,2845712,0,10,11.69,109,51.79,10.06,0.00,0.00,61.850,9.31,4.20,0.00,0.00,13.510
ATLA,21106284,0,0,1.88,0,0.40,1.40,0.00,0.00,1.800,0.00,0.00,0.00,0.00,0.000
BALDER,43562,0,14,0.00,263,106.87,3.25,0.00,0.00,110.120,36.18,1.24,0.00,0.00,37.420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YTTERGRYTA,4973114,1,0,7.39,0,0.29,2.22,0.41,0.00,3.289,0.00,0.00,0.00,0.00,0.000
ÆRFUGL,33310197,0,0,63.00,13,5.90,37.04,4.15,0.00,50.825,4.65,31.61,3.52,0.00,42.948
ØST FRIGG,43576,0,0,21.37,0,0.00,9.22,0.00,0.07,9.290,0.00,0.00,0.00,0.00,0.000


After extracting and merging as much data as we could, we next wanted to add the data that we had to extract ourselves.

For now, this data was:

- Average Core Photos
- Number of Completion logs
- Number of Wells per Field

In [4]:
# Create the column to store the information
main_file["Avg Core Photos"] = 0

# Read in the file containing the information to insert
file_name = extension + "core_photos_data.xlsx"
core_photos = read_excel(file_name)

# Here we go through all the information in the given file
# and we make sure to insert that information into the right
# row in the main file.
for i in range(0, len(core_photos)):
    cur_photo_id = int(core_photos.at[i, "NPDID Field"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_photo_id == cur_field_id):
            main_file.loc[[cur_field], ["Avg Core Photos"]] = core_photos.at[i, "Avg # of photos per field"]



In [5]:
# Similar to the previous cell

main_file["Completion logs"] = 0

file_name = extension + "completion_logs_per_field.xlsx"
completion_logs = read_excel(file_name)
for i in range(0, len(completion_logs)):
    cur_comple_id = int(completion_logs.at[i, "Field ID"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_comple_id == cur_field_id):
            main_file.loc[[cur_field], ["Completion logs"]] = completion_logs.at[i, "# of completion logs"]


In [6]:
# Same as the previous cell
main_file["# of Wells"] = 0

file_name = extension + "Number_of_well_logs_per_field_sonya.xlsx"
num_wells = read_excel(file_name)
for i in range(0, len(num_wells)):
    cur_well_id = int(num_wells.at[i, "NPDID field"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_well_id == cur_field_id):
            main_file.loc[[cur_field], ["# of Wells"]] = num_wells.at[i, "Frequency"]

Now we need to do some more interesting things for the different dates.

After reading in the file containing the date information for the different fields, we need to parse out the month and year from the given dates along with finding out the duration of each of the different phases a field goes through.

From the information that we read we are given a given fields Experimental start date, Developmental Start date, and Developmental End date. We first wanted to parse each of the dates into the month and the year and save them in separate columns.

Next from those dates we wanted to calculate how long a field was in it's experimental phase and it's developmental phase. The assumption that we use here is that

- 1) A field is classified as experimental from the drill date of it's first experimental well to that field's first developmental well is drilled.
- 2) A field is classified as developmental from the field's first developmental well is drilled to the completion of that field's last developmental well.

An issue that arrises here is that there were some experiemental wells that were active during the field was in it's developmental phase.

Another issue that arrises here when trying to calculate how long the phases were, was that some wells are either purely experiemental, meaning no developmental wells were ever drilled, or purely developmental, meaning no experiemental wells were ever drilled.

The way we handled these cases was by just finding the last completed well in which ever case the field is in. For example, if a field only contains experiemental wells, the development duration would be marked as 0 and the experimental phase would be denoted as the length of time between the first experimental well was drilled to the last completed experimental well. The same logic would be applied for developmental duration only dealing with the developmental wells and not experimental.

In [7]:
# Reading in the date information

file_name = extension + "Exp_start_Dev_start_and_end_spreadsheet.xlsx"
exp_dev_dates = read_excel(file_name)

In [8]:
# Creating the columns to hold the information
main_file["Exp start year"] = 0
main_file["Exp start month"] = 0

# The program will go through the file that contains the date information
# and try to line it up with the right field.
# If it encounters a valid date, anything other than 00.00.0000, then it would
# parse the month and year out and store it.
for i in range(0, len(exp_dev_dates)):
    cur_well_id = int(exp_dev_dates.at[i, "NPDID"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_well_id == cur_field_id):
            cur_date = exp_dev_dates.at[i, "Experiemental Start"]
            if(not(cur_date == "00.00.0000")):
                sep_date = cur_date.split(".")
                main_file.at[cur_field, "Exp start year"] = int(sep_date[2])
                main_file.at[cur_field, "Exp start month"] = int(sep_date[1])

In [9]:
# Same as the previous cell

main_file["Dev start year"] = 0
main_file["Dev start month"] =  0

for i in range(0, len(exp_dev_dates)):
    cur_well_id = int(exp_dev_dates.at[i, "NPDID"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_well_id == cur_field_id):
            cur_date = exp_dev_dates.at[i, "Developmental Start"]
            if(not(cur_date == "00.00.0000")):
                sep_date = cur_date.split(".")
                main_file.at[cur_field, "Dev start year"] = int(sep_date[2])
                main_file.at[cur_field, "Dev start month"] = int(sep_date[1])

In [10]:
# Same as the previous cell

main_file["Dev end year"] = 0
main_file["Dev end month"] =  0

for i in range(0, len(exp_dev_dates)):
    cur_well_id = int(exp_dev_dates.at[i, "NPDID"])
    for cur_field in main_file.index:
        cur_field_id = int(main_file.at[cur_field, "NPDID field"])
        if(cur_well_id == cur_field_id):
            cur_date = exp_dev_dates.at[i, "Developmental End"]
            if(not(cur_date == "00.00.0000")):
                sep_date = cur_date.split(".")
                main_file.at[cur_field, "Dev end year"] = int(sep_date[2])
                main_file.at[cur_field, "Dev end month"] = int(sep_date[1])

In [11]:
# Creating the column to store the durations

main_file["Exp Duration"] = 0
main_file["Dev Duration"] = 0

# The loop will optain the different dates for the current field
for cur_field in main_file.index:
    exp_month = main_file.at[cur_field, "Exp start month"]
    exp_year = main_file.at[cur_field, "Exp start year"]
    dev_smonth = main_file.at[cur_field, "Dev start month"]
    dev_syear = main_file.at[cur_field, "Dev start year"]
    dev_emonth = main_file.at[cur_field, "Dev end month"]
    dev_eyear = main_file.at[cur_field, "Dev end year"]
    
    # Calculating the duration's for a given phase in months
    exp_dur = (dev_syear - exp_year) * 12 + (dev_smonth - exp_month)
    dev_dur = 0
    
    # Making sure that there is a valid date for the development end date
    # If there is, then calculate normally, if not then ignore.
    if(not(dev_emonth == 0) and not(dev_eyear == 0)):
        dev_dur = (dev_eyear - dev_syear) * 12 + (dev_emonth - dev_smonth)
    
    # Saving the information.
    main_file.at[cur_field, "Exp Duration"] = exp_dur
    main_file.at[cur_field, "Dev Duration"] = dev_dur

In [12]:
main_file

,NPDID field,Orig. inplace ass. liquid \n[mill Sm3],Orig. inplace ass. gas \n[bill Sm3],Orig. inplace free gas \n[bill Sm3],Orig. inplace oil \n[mill Sm3],Orig. recoverable oil \n[mill Sm3],Orig. recoverable gas \n[bill Sm3],Orig. recoverable NGL \n[mill tonn],Orig. recoverable cond. \n[mill Sm3],Orig. recoverable oil eq. \n[mill Sm3 o.e],...,Completion logs,# of Wells,Exp start year,Exp start month,Dev start year,Dev start month,Dev end year,Dev end month,Exp Duration,Dev Duration
Field name,,,,,,,,,,,,,,,,,,,,,
ALBUSKJELL,43437,0,64,0.00,56,7.35,15.53,0.99,0.00,24.761,...,0,26,1972,7,1978,6,1986,6,71,96
ALVE,4444332,3,1,13.50,3,2.39,9.08,1.41,0.00,14.149,...,0,8,1990,5,2009,1,2016,7,224,90
ALVHEIM,2845712,0,10,11.69,109,51.79,10.06,0.00,0.00,61.850,...,0,15,1974,10,2006,5,2019,9,379,160
ATLA,21106284,0,0,1.88,0,0.40,1.40,0.00,0.00,1.800,...,0,5,2010,8,2012,6,2012,6,22,0
BALDER,43562,0,14,0.00,263,106.87,3.25,0.00,0.00,110.120,...,0,7,1970,4,1996,5,2016,4,313,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YTTERGRYTA,4973114,1,0,7.39,0,0.29,2.22,0.41,0.00,3.289,...,0,2,2007,5,2008,10,2008,10,17,0
ÆRFUGL,33310197,0,0,63.00,13,5.90,37.04,4.15,0.00,50.825,...,0,4,2000,5,2010,9,2020,3,124,114
ØST FRIGG,43576,0,0,21.37,0,0.00,9.22,0.00,0.07,9.290,...,0,5,1973,8,1987,7,1988,3,167,8


The next piece of information that we wanted to do was include the monthly production results of the fields and combine them into a readable file.

So the way that this file will be organized is that Month 1 will represent the first month that a field was in operation. This means that Month 1 for one field may represent May 1993 while another field's Month 1 will represent June 2002.

In [13]:
# Create the place holders for all the potential production months for a field.
for i in range(1,584):
    curMonth = "Oil - Month " + str(i)
    main_file[curMonth] = np.nan
    
for i in range(1,584):
    curMonth = "Gas - Month " + str(i)
    main_file[curMonth] = np.nan
main_file

,NPDID field,Orig. inplace ass. liquid \n[mill Sm3],Orig. inplace ass. gas \n[bill Sm3],Orig. inplace free gas \n[bill Sm3],Orig. inplace oil \n[mill Sm3],Orig. recoverable oil \n[mill Sm3],Orig. recoverable gas \n[bill Sm3],Orig. recoverable NGL \n[mill tonn],Orig. recoverable cond. \n[mill Sm3],Orig. recoverable oil eq. \n[mill Sm3 o.e],...,Gas - Month 574,Gas - Month 575,Gas - Month 576,Gas - Month 577,Gas - Month 578,Gas - Month 579,Gas - Month 580,Gas - Month 581,Gas - Month 582,Gas - Month 583
Field name,,,,,,,,,,,,,,,,,,,,,
ALBUSKJELL,43437,0,64,0.00,56,7.35,15.53,0.99,0.00,24.761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALVE,4444332,3,1,13.50,3,2.39,9.08,1.41,0.00,14.149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALVHEIM,2845712,0,10,11.69,109,51.79,10.06,0.00,0.00,61.850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLA,21106284,0,0,1.88,0,0.40,1.40,0.00,0.00,1.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BALDER,43562,0,14,0.00,263,106.87,3.25,0.00,0.00,110.120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YTTERGRYTA,4973114,1,0,7.39,0,0.29,2.22,0.41,0.00,3.289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ÆRFUGL,33310197,0,0,63.00,13,5.90,37.04,4.15,0.00,50.825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ØST FRIGG,43576,0,0,21.37,0,0.00,9.22,0.00,0.07,9.290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# reading in the monthly production values for gas and oil
monthly = pd.read_excel(extension + "field_production_monthly.xlsx")
monthly

,Field (Discovery),Year,Month,Net - oil \n[mill Sm3],Net - gas \n[bill Sm3],Net - NGL \n[mill Sm3],Net - condensate \n[mill Sm3],Net - oil equivalents \n[mill Sm3],Produced water in field \n[mill Sm3],NPDID information carrier
0,1/5-2 Flyndre,2017,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43838
1,1/5-2 Flyndre,2017,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43838
2,1/5-2 Flyndre,2017,3,0.000320,0.000000,0.000003,0.000000,0.000323,0.000000,43838
3,1/5-2 Flyndre,2017,4,0.002589,0.000000,0.000031,0.000000,0.002620,0.000001,43838
4,1/5-2 Flyndre,2017,5,0.002123,0.000000,0.000031,0.000000,0.002154,0.000001,43838
...,...,...,...,...,...,...,...,...,...,...
20779,AASTA HANSTEEN,2019,8,0.000000,0.675740,0.000000,0.019321,0.695061,0.002626,23395946
20780,AASTA HANSTEEN,2019,9,0.000000,0.200317,0.000000,0.003316,0.203633,0.000510,23395946
20781,AASTA HANSTEEN,2019,10,0.000000,0.742341,0.000000,0.017377,0.759718,0.001451,23395946
20782,AASTA HANSTEEN,2019,11,0.000000,0.670464,0.000000,0.014567,0.685031,0.001190,23395946


In [15]:
# This is the loop that will put the correct monthly production values into the main file.

# Firstly the loop will loop through all the fields that we are recording.
for cur_field in main_file.index:
    # Takes note of the NPDID
    cur_field_id = int(main_file.at[cur_field, "NPDID field"])   
    
    # pulls out the monthly production related to the current fields NPDID number
    cur_field_prod = monthly[monthly['NPDID information carrier'] == cur_field_id]
    cur_field_prod = cur_field_prod.reset_index()
    
    # Next it proceeds to put all the production data related
    # to that NPDID number into the main file 
    for j in range(0, len(cur_field_prod)):
        cur_oil_prod = cur_field_prod['Net - oil \n[mill Sm3]'][j]
        cur_gas_prod = cur_field_prod['Net - gas \n[bill Sm3]'][j]
        new_month = j + 1
        oil_month_key = "Oil - Month " + str(new_month)
        gas_month_key = "Gas - Month " + str(new_month)
        main_file.loc[[cur_field], [oil_month_key]] = cur_oil_prod
        main_file.loc[[cur_field], [gas_month_key]] = cur_gas_prod

In [16]:
main_file

,NPDID field,Orig. inplace ass. liquid \n[mill Sm3],Orig. inplace ass. gas \n[bill Sm3],Orig. inplace free gas \n[bill Sm3],Orig. inplace oil \n[mill Sm3],Orig. recoverable oil \n[mill Sm3],Orig. recoverable gas \n[bill Sm3],Orig. recoverable NGL \n[mill tonn],Orig. recoverable cond. \n[mill Sm3],Orig. recoverable oil eq. \n[mill Sm3 o.e],...,Gas - Month 574,Gas - Month 575,Gas - Month 576,Gas - Month 577,Gas - Month 578,Gas - Month 579,Gas - Month 580,Gas - Month 581,Gas - Month 582,Gas - Month 583
Field name,,,,,,,,,,,,,,,,,,,,,
ALBUSKJELL,43437,0,64,0.00,56,7.35,15.53,0.99,0.00,24.761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALVE,4444332,3,1,13.50,3,2.39,9.08,1.41,0.00,14.149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALVHEIM,2845712,0,10,11.69,109,51.79,10.06,0.00,0.00,61.850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLA,21106284,0,0,1.88,0,0.40,1.40,0.00,0.00,1.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BALDER,43562,0,14,0.00,263,106.87,3.25,0.00,0.00,110.120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YTTERGRYTA,4973114,1,0,7.39,0,0.29,2.22,0.41,0.00,3.289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ÆRFUGL,33310197,0,0,63.00,13,5.90,37.04,4.15,0.00,50.825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ØST FRIGG,43576,0,0,21.37,0,0.00,9.22,0.00,0.07,9.290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Export the final file
main_file.to_csv("main_file_final.csv")